In [141]:
import sys, json, pprint, urllib
import pandas as pd
from datetime import datetime

In [142]:
INPUT_FILE = 'raw-geo-earthquake-tweets.jsonl'
#INPUT_FILE = '../epic-earthquake-tweets.jsonl'
OUTPUT_FILE = 'earthquake-tweets.geojsonl'

## Parse out Earthquake Tweets and Visualize them

In [143]:
day_counter = {}
def update_day_counter(dt):
        
    date = dt.date()
    
    if date in day_counter:
        day_counter[date]+=1;
    else:
        day_counter[date]= 1

In [144]:
#Wed Apr 13 02:46:23 +0000 2016
"%a %b %d %X %z %Y"

def createProps(feature, tweet):
    dt = datetime.strptime(tweet['created_at'], "%a %b %d %X %z %Y")
    feature['properties'] = {
#             'orig': tweet['created_at'],
#             'datetime' : datetime.strptime(tweet['created_at'], "%a %b %d %X %z %Y"),
             'timestamp' : int(dt.timestamp()),
             'user' : tweet['user']['screen_name'],
             'text' : urllib.parse.quote_plus(tweet['text'])
    }
    
    update_day_counter(dt)

    #if 'entities' in tweet:
    #    if 'hashtags' in tweet['entities']:
    #        for idx, tag in enumerate(tweet['entities']['hashtags'][:5]):
    #            feature['properties']['hashtag_'+str(idx+1)] = tag['text'].lower()
    return feature

def makeFeature(t):
    f = {'type':'Feature'}
    if 'coordinates' in t:
        if t['coordinates'] is not None:
            f['geometry'] = t['coordinates']
            return createProps(f,t)
    elif 'geo' in t:
        if 'geo' is not None:
            f['geometry'] = {'type':'Point', 'coordinates': reversed(t['geo']['coordinates'])}
            return createProps(f,t)
    return None

In [ ]:
count = 0;
geo = 0;
limit = 999999999;
features = []
with open(INPUT_FILE,'r') as inFile:
    with open(OUTPUT_FILE,'w') as outFile:
        for line in inFile:
            count += 1;
            t = makeFeature(json.loads(line.strip()))
            if t:
                features.append(t)
                outFile.write(json.dumps(t)+"\n")
                geo+=1;
            if count > limit:
                break
            if count%100==0:
                sys.stderr.write("\r{0} tweets processed, {1} with geo".format(count, geo))
len(features)

2134700 tweets processed, 754427 with geo

## Create CSV for Timeline

In [146]:
with open('docs/earthquakes-meta.csv','w') as tweet_count_meta:
    tweet_count_meta.write("date,count\n")
    
    for date, count in sorted([(x[0], x[1]) for x in day_counter.items()], key=lambda x: x[0]):
    
        string = "{0},{1}\n".format(date, count)         
        tweet_count_meta.write( string )
        #print(string, end="")

In [123]:
json.dump({'type':'FeatureCollection','features':features}, open("/data/www/jennings/earthquakes.geojson",'w'))